In [1]:
import numpy as np
import warnings
import sys
import yaml
import os
import traceback
import xarray as xr
import pandas as pd
import numpy as np
import echopype as ep

from yaml.loader import SafeLoader
from functions import find_waves, process_data, save_data, find_bottom, data_to_images, find_fish_median, medianfun, find_layer, remove_vertical_lines, clean_times

warnings.filterwarnings("ignore")


# Load all params from yaml-file
with open('params.yaml', 'r') as f:
    params = list(yaml.load_all(f, Loader=SafeLoader))


# Remove already processed files  
csv_path = "data_out/csv"
img_path = "data_out/img"
sonar_depth = 0.53


file = "SLUAquaSailor2020V3-Phase0-D20230512-T011741-1.raw"
path = '/mnt/pr_SLU/pr_SLU/Sailor/raw/2023'

In [2]:
raw_echodata = ep.open_raw(f'{path}/{file}', sonar_model="EK80")
raw_echodata

<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
│   └── NMEA: contains information specific to the NMEA protocol.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains complex backscatter data and other beam or channel-specific data.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

In [5]:
ds_Sv_raw = ep.calibrate.compute_Sv(
        raw_echodata,
        env_params = params[0]['env_params'],
        cal_params = params[0]['cal_params'],
        waveform_mode="BB",
        encode_mode="complex",
    )

ds_MVBS = ep.commongrid.compute_MVBS(
    ds_Sv_raw, # calibrated Sv dataset
    range_meter_bin=params[0]['bin_size'], # bin size to average along range in meters
    ping_time_bin='2S' # bin size to average along ping_time in seconds
)

ds_MVBS

<xarray.Dataset>
Dimensions:            (channel: 1, ping_time: 26, echo_range: 1000)
Coordinates:
  * channel            (channel) <U30 'EKA 266980-07 ES200-7CDK-Split'
  * ping_time          (ping_time) datetime64[ns] 2023-05-12T01:17:42 ... 202...
  * echo_range         (echo_range) float64 0.0 0.1 0.2 0.3 ... 99.7 99.8 99.9
Data variables:
    Sv                 (channel, ping_time, echo_range) float64 nan ... -153.4
    frequency_nominal  (channel) float64 2e+05
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.8.1
    processing_time:              2024-05-16T07:53:41Z
    processing_function:          commongrid.compute_MVBS